In [ ]:
import logging
import time
from datetime import datetime, date, timedelta
from time import sleep

import requests
from fastapi import APIRouter, Depends
from fastapi.encoders import jsonable_encoder
from fastapi_utils.cbv import cbv
from sqlalchemy import func, select, case, delete, insert
from sqlalchemy.ext.asyncio import AsyncSession
from urllib.parse import urlencode

from common.db import db_krx
from models.models_krx import KindIssue
from models.models_krx import KindIssueTemporary
from models.models_krx import KoreaStock
from routers.scrap.krx.stock.issue import KrxStockIssue

import json
import feedparser

from dateutil.parser import parse


router = APIRouter()
logger = logging.getLogger('uvicorn')

@cbv(router=router)
class KrxDisclosure:
  db:AsyncSession = Depends(db_krx.get_db)
  krxStockIssue:KrxStockIssue = Depends(KrxStockIssue)

  url_data='http://kind.krx.co.kr:80/disclosure/rsstodaydistribute.do?method=searchRssTodayDistribute&repIsuSrtCd=&mktTpCd=0&searchCorpName=&currentPageSize=15'
  
  #크롤링이 잘 작동하는지 확인하는 get router 구축

  @router.get('/krx_disclosure',
    summary='오늘의 공시 데이터 크롤링',
  )
  async def get_kind_krx(self):

    try:
      start=datetime.now()
      res = requests.get(self.url_data)
    #print(res)

      data = feedparser.parse(res.text)

      data=data['entries']
      # data_count=0
      # print(data)
      # published=datetime.fromtimestamp(time.mktime(data[0].published_parsed))

    
      # print(published)
      return data
    except Exception as e:
      logger.exception(e)



    # 여기까지는 데이터 잘 받는 것 확인.

    
  # 업데이트 후 데이터 적재하는 코드 
  
  @router.post('/disclosureup',
    summary='오늘의 공시 정보 업데이트')

  async def updated_disclosure(self):


      data = await self.get_kind_krx()
      # data= data ['entries']
      data_count=0
    
      print(data)
      for items in data:
        
        # if '공시' in items.title:
        #   self.insert_item(items,'공시')
        if '권리락' in items.title:
          self.insert_item(items,'권리락')
        if '배당락' in items.title:
          self.insert_item(items,'배당락')
        if '합병' in items.title:
          self.insert_item(items,'합병')
        if '액면분할' in items.title:
          self.insert_item(items,'액면분할')
        if '유상증자' in items.title:
          self.insert_item(items,'유상증자')
        if '무상증자' in items.title:
          self.insert_item(items,'무상증자')
        if '유무상증자' in items.title:
          self.insert_item(items,'유무상증자')
        if '상장폐지' in items.title:
          self.insert_item(items,'상장폐지')

        data_count+=1

        


      # time=datetime.now()-start
      # logger.info(time)

      try:
        await self.db.commit()
        
        return {'data_count' : data_count}
      except Exception:
        await self.db.rollback()
        logger.exception('kind_krx 저장오류')
        return None
  
  #   try:
  #     await self.db.commit()
     
  #     return {'data_count':data_count, 'time':time}
      
  #   except Exception:
  #     logger.exception('kind_krx 저장 오류')
  #     return None
    

  def insert_item(self,item,type):
    if ']' in item.author:
      asset_market,author=item.author.split(']')
    else:
      asset_market=''
      author=item.author


    #리팩토링하기  
    # print(datetime.fromtimestamp(time.mktime(item.published_parsed)))
    published=datetime.fromtimestamp(time.mktime(item.published_parsed))
    
    updated=datetime.fromtimestamp(time.mktime(item.updated_parsed))
   
    asset_market=self.change_asset_market(asset_market.replace('[',''))
    title=item.title.strip('[코]''[유]''[넥]')

    item = KindIssue(**{
      'issue_type': type,
      'title': title,
      'link': item.link,
      'published': published,
      'asset_market': asset_market,
      'author': author,
      'tags_term': item.tags[0]['term'],
      'language': item.language,
      'updated': updated, 
    })

    self.db.add(item)



    item2 = KindIssueTemporary(**{
      'issue_type': type,
      'title': title,
      'asset_market': asset_market,
      'author': author,
    })
  
    self.db.add(item2)




  def change_str_to_datetime(self,date):

    item=date.split()
    month=datetime.strptime(item[2], "%b").month

    d=item[3]+'-'+str(month)+'-'+item[1]+' '+item[4]
    new_date=datetime.strptime(d, '%Y-%m-%d %H:%M:%S')

    return new_date

  def change_asset_market(self,item):
    if item=='코':
      item='KOSDAQ'
    if item=='유':
      item='KOSPI'
    if item=='넥':
      item='KONEX'
    return item


  async def get_disclosure_dbdata(self):

    try:
      q = select(KindIssue)
      rs = await self.db.execute(q)
      return rs.scalars().all()

    except Exception as e:
      print(e)
      


  @router.post('/issuedisclosure',
    summary=''
  
  )

#krx_issue와 disclosure 비교 후 해당 값 추가 

  async def krx_comparsion(self):
    
    print("start")
    try:
      issueorigin:list[KoreaStock] = await self.krxStockIssue.get_origin_data()
      disclosureorigin:list[KindIssue] = await self.get_disclosure_dbdata()

      for d in disclosureorigin :
        for i in issueorigin:
          if d.author == i.ticker_name:
            if '권리락' in d.issue_type:
              i.right=datetime.today()
            elif '배당락' in d.issue_type:
              i.allocation=datetime.today()
            elif '합병' in d.issue_type:
              i.merger=datetime.today()
            elif '액면분할' in d.issue_type:
              i.stock_split=datetime.today()
            elif '유상증자' in d.issue_type:
              i.paid=datetime.today()
            elif '무상증자' in d.issue_type:
              i.free=datetime.today()
            elif '유무상증자' in d.issue_type:
              i.paidfree=datetime.today()
            elif '상장폐지' in d.issue_type:
              i.delisting=datetime.today()
            
      
      await self.db.commit()

    
    except Exception as e:
      print(f"error : {e.with_traceback()}")
      return None
            
